Importando as bibliotecas necessárias.

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

Extração dos links, utilizando BS4 e Selenium, site base contém diversos links dos imóveis e um botão "Ver Mais"
em que utilizei Selenium (linha 20 - 22) para continuar avançando.

Pode ser útil alterar os parãmetros em 'WebDriverWait(driver, 2)' e 'time.sleep(2)' dependendo da potência
do computador do usuário desse código.

In [60]:
lista_links = []

def extrair_links(soup):
    linked = soup.find_all("a", class_='sc-1d0oyoa-0 bfDoiv')
    return [link.get('href') for link in linked]

url = f'https://www.quintoandar.com.br/comprar/imovel/rio-de-janeiro-rj-brasil'

chrome_options = Options()
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get(url)
    wait = WebDriverWait(driver, 5)
    for i in range(10):
        try:
            ver_mais_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Ver mais"]')))
            ver_mais_button.click()
            time.sleep(10)
        except:
            break
    final_html = driver.page_source
    soup_final = BeautifulSoup(final_html, 'html.parser')
    lista_links = extrair_links(soup_final)

finally:
    driver.quit()

Criando as listas em que os dados serão armazenados.

In [61]:
lista_preços = []
lista_elem = []
lista_item = []
lista_condominio = []
lista_id = []
lista_iptus = []
lista_endereço1 = []
lista_endereço2 = []
lista_tempos = []
lista_tempos_dias = []
lista_tipos = []

Criando as funções que serão utilizadas, todas elas possuem condicionais que garantem que, mesmo que o elemento
não seja encontrado, ele ainda retorne para a lista "Not Informed", mantendo a ordem das listas.

In [62]:
def id(soup):
    buscando = soup.find('a',{'aria-current': 'page'})
    if buscando:
            id = buscando.find('span', class_ = 'CozyTypography xih2fc Bm9DLt _1tBHcU')
            lista_id.append(id.text.strip().replace("Imóvel","")) if id else lista_id.append("Not Informed")
    else:
        lista_id.append("Not Informed")
    return lista_id

def tipo(soup):
    buscando = soup.find_all('h1', class_ = 'CozyTypography xih2fc EKXjIf wIyEP2 _8JKqPG r4Q8xM')
    if buscando:
        for encontrado in buscando:
            cleaning = encontrado.text.split()
            lista_tipos.append(cleaning[0]) if cleaning else lista_tipos.append("Not Informed")
    else:
        lista_tipos.append("Not Informed")
    return lista_tipos

def precos(soup):
    buscando = soup.find("p", class_='CozyTypography xih2fc wIyEP2 _8JKqPG r4Q8xM')
    if buscando:
            cleaning = buscando.text.strip().replace("\xa0","")
            lista_preços.append(cleaning.replace("R$",''))
    else:
        lista_preços.append("Not Informed")
    return lista_preços

def enderecos_pt1(soup):
    buscando = soup.find_all("small", class_='CozyTypography xih2fc pwAPLE')
    if buscando:
        for endereço in buscando:
            lista_endereço1.append(endereço.text.strip().replace("\xa0",""))
    else:
        lista_endereço1.append("Not Informed")
    return lista_endereço1

def enderecos_pt2(soup):
        buscando = soup.find("h4", class_='CozyTypography xih2fc EqjlRj')
        if buscando:
                lista_endereço2.append(buscando.text.strip())
        else:
                lista_endereço2.append("Not Informed")
        return lista_endereço2

def elementos_base(soup):
        cont = 0
        elemexists = soup.find_all("p", class_ = "CozyTypography xih2fc EKXjIf Ci-jp3")
        if elemexists:
            for elem_end in elemexists:
                if cont == 7:
                    break
                else:
                    lista_elem.append(elem_end.text.strip().replace("\xa0",""))
                    cont = cont + 1
        else:
            for i in range(7):
                lista_elem.append("Not Informed")
        return lista_elem

def itens(soup):
    sublist = []
    item = soup.find_all("div", class_='MuiBox-root mui-117mro9')
    if item:
        for elem in item:
            exists = elem.find_all("p", class_ = 'CozyTypography xih2fc Ci-jp3 p4uM-0')
            if exists:
                for elemnow in exists:
                    sublist.append(elemnow.text.strip().replace("\xa0",""))
    else:
        sublist.append("Not Informed")
    cleaning = ', '.join(sublist)
    return cleaning

def condominio(soup):
    buscando = soup.find('span', class_ = 'CozyTypography sc-1jl0yev-0 cTsiPw xih2fc _72Hu5c Ci-jp3')
    if buscando:
            cleaning = buscando.text.strip().replace('\xa0','')
            lista_condominio.append(cleaning.replace("R$",""))
    else:
        lista_condominio.append("Not Informed")
    return lista_condominio

def iptu(soup):
    buscando = soup.find_all('span', class_="CozyTypography sc-1jl0yev-0 cTsiPw xih2fc _72Hu5c Ci-jp3")
    if buscando:
        for i in range(1, len(buscando), 2):
            iptu_text = buscando[i].text
            cleaned = iptu_text.replace("R$", "").replace("\xa0", "")
            cleaned1 = cleaned.replace("12x ","")
            lista_iptus.append(cleaned1)
    else:
        lista_iptus.append("Not Informed")
    return lista_iptus

def tempo(soup):
    buscando = soup.find('small',class_ = 'CozyTypography xih2fc Bm9DLt _72Hu5c _1tBHcU')
    if buscando:
        lista_tempos.append(buscando.text.strip())
    else:
        lista_tempos.append("Not Informed")
    return lista_tempos

def converte_dias(lista_tempos):
    for elem in lista_tempos:
        divisão = elem.split()
        if elem == "Not Informed":
            lista_tempos_dias.append("Not Informed")
        elif divisão[3] == 'mês' or divisão[3] == 'meses':
            lista_tempos_dias.append(int(divisão[2]) * 30)
        elif divisão[3] == 'horas' or divisão[3] == 'hora':
            lista_tempos_dias.append(round(int(divisão[2])/24,2))
        elif divisão[3] == 'dias' or divisão[3] == 'dia':
            lista_tempos_dias.append(int(divisão[2]))
        else:
            lista_tempos_dias.append(int(divisão[4])*365)
    return lista_tempos_dias

Iterando pela lista de links obtidos, e aplicando as funções mencionadas acima.

In [63]:
for links in lista_links:
    response = requests.get(links)
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    iptu(soup)
    id(soup)
    precos(soup)
    condominio(soup)
    elementos_base(soup)
    enderecos_pt1(soup)
    enderecos_pt2(soup)
    lista_item.append(itens(soup))
    tempo(soup)
    tipo(soup)

Populando mais listas derivadas da outra lista "Lista_Elem", que foi preparada de modo que, a cada 7 elementos
fosse relacionada à um novo link, além disso, também convertendo os "meses", "anos", "horas" para "dias".

In [ ]:
quartos = []
tamanho = []
metro = []
vagas = []
banheiros = []
andar = []
mobilia = []

for i in range(len(lista_elem)):
    if i % 7 == 0:
        quartos.append(lista_elem[i][0])
    elif i % 7 == 1:
        tamanho.append(lista_elem[i].replace("m²",''))
    elif i % 7 == 2:
        metro.append(lista_elem[i])
    elif i % 7 == 3:
        vagas.append(lista_elem[i][0])
    elif i % 7 == 4:
        banheiros.append(lista_elem[i][0])
    elif i % 7 == 5:
        andar.append(lista_elem[i])
    elif i % 7 == 6:
        mobilia.append(lista_elem[i])

converte_dias(lista_tempos)


Primeiro dataset com informações GERAIS.

In [65]:
descrição_imovel = {
    'ID': lista_id,
    'Tipo' : lista_tipos,
    'Descrição': lista_item,
    'Bairro' : lista_endereço1,
    'Logradouro' : lista_endereço2,
    'Publicação(Dias)' : lista_tempos_dias
}

colunas_numericas = ['ID','Publicação(Dias)']

df = pd.DataFrame(descrição_imovel)
df

,ID,Tipo,Descrição,Bairro,Logradouro,Publicação(Dias)
0,1027119,Apartamento,"Piscina privativa, Quartos e corredores com po...","Tanque, Rio de Janeiro",Rua Manuel Vieira,5.00
1,1263229,Casa,"Box, Varanda, Armários na cozinha, Quintal, So...","Piedade, Rio de Janeiro",Rua Almeida Nogueira,150.00
2,815144,Apartamento,,"Centro, Rio de Janeiro",Rua Alcântara Machado,0.17
3,1762586,Casa,"Box, Varanda, Armários nos banheiros, Armários...","Tanque, Rio de Janeiro",Estrada da Covanca,2.00
4,1743612,Casa,"Box, Varanda, Armários embutidos no quarto, Ar...","Tanque, Rio de Janeiro",Rua Jordão,30.00
...,...,...,...,...,...,...
127,622975,Studio,"Armários embutidos no quarto, Armários nos ban...","Catete, Rio de Janeiro",Rua Bento Lisboa,7.00
128,652678,Casa,,"Riachuelo, Rio de Janeiro",Rua Marechal Bittencourt,30.00
129,1767937,Casa,"Box, Varanda, Quintal, Área de serviço","Jacarepaguá, Rio de Janeiro",Estrada da Ligação,18.00
130,1777905,Apartamento,"Box, Varanda, Armários nos banheiros, Armários...","Benfica, Rio de Janeiro",Rua Senador Bernado Monteiro,4.00


 Segundo dataset com informações de CUSTOS

In [66]:
descrição_valores = {
    'ID': lista_id,
    'Preço' : lista_preços,
    'Condomínio' : lista_condominio,
    'IPTU' : lista_iptus,
}

df1 = pd.DataFrame(descrição_valores)
df1['ID'] = pd.to_numeric(df1['ID'], downcast='integer',errors = "coerce")
df1

,ID,Preço,Condomínio,IPTU
0,1027119,150.000,450,0
1,1263229,250.000,0,459
2,815144,330.000,466,66
3,1762586,450.000,70,0
4,1743612,245.000,250,0
...,...,...,...,...
127,622975,270.000,453,21
128,652678,200.000,0,0
129,1767937,200.000,0,8
130,1777905,295.000,310,0


Terceiro dataset com informações INTERNAS

In [67]:
descrição_especificas = {
    'ID': lista_id,
    'Quartos': quartos,
    'Área': tamanho,
    'Proximidade com Mêtro': metro,
    'Vagas': vagas,
    'Banheiros': banheiros,
    'Andares': andar,
    'Mobiliagem': mobilia
}

colunas_numericas3 = ['ID','Quartos','Área','Vagas','Banheiros']

df2 = pd.DataFrame(descrição_especificas)
df2[colunas_numericas3] = df2[colunas_numericas3].apply(pd.to_numeric,errors = 'coerce')
df2

,ID,Quartos,Área,Proximidade com Mêtro,Vagas,Banheiros,Andares,Mobiliagem
0,1027119,2,55,Sem metrô próx.,1.0,1,Até 3° andar,Sem mobília
1,1263229,4,441,Próx. ao metrô,3.0,3,-,Sem mobília
2,815144,1,56,Próx. ao metrô,NaN,1,4° a 7° andar,Sem mobília
3,1762586,4,450,Sem metrô próx.,2.0,4,-,Sem mobília
4,1743612,3,21,Sem metrô próx.,3.0,4,-,Sem mobília
...,...,...,...,...,...,...,...,...
127,622975,1,32,Próx. ao metrô,NaN,1,Até 3° andar,Sem mobília
128,652678,2,70,Próx. ao metrô,NaN,1,-,Sem mobília
129,1767937,1,180,Sem metrô próx.,3.0,1,Até 3° andar,Sem mobília
130,1777905,2,60,Próx. ao metrô,1.0,1,Até 3° andar,Sem mobília


Quarto dataset com a EXPLOSÃO da coluna Descrição

In [68]:
descrição_itens = {
    'ID': lista_id,
    'Descrição': lista_item
}

df3 = pd.DataFrame(descrição_itens)

df3["Itens"] = df3["Descrição"].str.split(", ")
df_exploded = df3.explode("Itens")
df_exploded["Descrição"] = df_exploded["Itens"]

df_exploded.drop("Itens", axis=1, inplace=True)
df_exploded

,ID,Descrição
0,1027119,Piscina privativa
0,1027119,Quartos e corredores com portas amplas
0,1027119,Área de serviço
1,1263229,Box
1,1263229,Varanda
...,...,...
130,1777905,Área de serviço
131,529139,Box
131,529139,Armários nos banheiros
131,529139,Armários na cozinha
